<a href="https://colab.research.google.com/github/AdamPeetz/PlaneGAN/blob/main/MSDS696_ACGAN_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from numpy import zeros
from numpy import ones
from numpy import expand_dims
from numpy.random import randn, random
from numpy.random import randint
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization, Dropout
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Activation
from keras.layers import Concatenate
from keras.initializers import RandomNormal
from matplotlib import pyplot
import tensorflow as tf
import cv2
import numpy as np
import pathlib
from matplotlib import pyplot
import matplotlib as plt

import os, shutil 
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
# define the standalone discriminator model
def define_discriminator(in_shape=(128,128,3), n_classes=7):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# image input
	in_image = Input(shape=in_shape)
	# normal
	fe = Conv2D(64, (3,3), padding='same', kernel_initializer=init)(merge)
	fe = LeakyReLU(alpha=0.2)(fe)
	# downsample 64x64
	fe = Conv2D(96, (3,3), strides=(2,2), kernel_initializer=init, padding='same')(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	fe = Dropout(0.4)(fe)
	# downsample 32x32
	fe = Conv2D(128, (3,3), strides=(2,2), kernel_initializer=init, padding='same')(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	fe = Dropout(0.4)(fe)
 	# downsample 16x16
	fe = Conv2D(192, (3,3), strides=(2,2), kernel_initializer=init, padding='same')(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	fe = Dropout(0.4)(fe)
 	# downsample 8x8
	fe = Conv2D(256, (3,3), strides=(2,2), kernel_initializer=init, padding='same')(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	fe = Dropout(0.4)(fe)
	# downsample 4x4
	fe = Conv2D(256, (3,3), strides=(2,2), kernel_initializer=init, padding='same')(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	fe = Dropout(0.4)(fe)
	# flatten feature maps
	fe = Flatten()(fe)
	# real/fake output
	out1 = Dense(1, activation='sigmoid')(fe)
	# class label output
	out2 = Dense(n_classes, activation='softmax')(fe)
	# define model
	model = Model(in_image, [out1, out2])
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
	return model

In [9]:
# define the standalone generator model
def define_generator(latent_dim, n_classes=7):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# label input
	in_label = Input(shape=(1,))
	# embedding for categorical input
	li = Embedding(n_classes, 50)(in_label)
	# linear multiplication
	n_nodes = 7 * 7
	li = Dense(n_nodes, kernel_initializer=init)(li)
	# reshape to additional channel
	li = Reshape((4, 4, 1))(li)
	# image generator input
	in_lat = Input(shape=(latent_dim,))
	# foundation for 7x7 image
	n_nodes = 256 * 4 * 4
	gen = Dense(n_nodes, kernel_initializer=init)(in_lat)
	gen = LeakyReLU(alpha=0.2)(gen)
	gen = Reshape((4, 4, 256))(gen)
	# merge image gen and label input
	merge = Concatenate()([gen, li])
	# upsample to 8x8
	gen = Conv2DTranspose(1024, (4,4), strides=(2,2), kernel_initializer=init, padding='same')(merge)
	gen = LeakyReLU(alpha=0.2)(gen)
	# upsample to 16x16
	gen = Conv2DTranspose(512, (4,4), strides=(2,2), kernel_initializer=init, padding='same')(gen)
	gen = LeakyReLU(alpha=0.2)(gen)
	# upsample to 32x32
	gen = Conv2DTranspose(264, (4,4), strides=(2,2), kernel_initializer=init, padding='same')(gen)
	gen = LeakyReLU(alpha=0.2)(gen)
	# upsample to 64x64
	gen = Conv2DTranspose(128, (4,4), strides=(2,2), kernel_initializer=init, padding='same')(gen)
	gen = LeakyReLU(alpha=0.2)(gen)
	# upsample to 128x128
	gen = Conv2DTranspose(64, (4,4), strides=(2,2), kernel_initializer=init, padding='same')(gen)
	gen = LeakyReLU(alpha=0.2)(gen)
	# output
	out_layer = Conv2D(3, (3,3), activation='tanh', padding='same')(gen)
	# define model
	model = Model([in_lat, in_label], out_layer)
	return model

In [10]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	for layer in d_model.layers:
		if not isinstance(layer, BatchNormalization):
			layer.trainable = False
	# connect the outputs of the generator to the inputs of the discriminator
	gan_output = d_model(g_model.output)
	# define gan model as taking noise and label and outputting real/fake and label outputs
	model = Model(g_model.input, gan_output)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
	return model

In [13]:
# load custom image dataset, developed from tensorflow documentation
def load_custom_image_set(dataroot,resizedim,directorylevels):
    #image parse function
    def parse_image(filename):
        parts = tf.strings.split(filename, os.sep)
        label = parts[-2]
        label = int(label)
        image = tf.io.read_file(filename)
        image = tf.io.decode_jpeg(image)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.image.resize(image, [resizedim,resizedim])
        image = (image * 2) -1
        return image, label
    
    data_root = pathlib.Path(dataroot)
    if directorylevels == 1:
      list_ds = tf.data.Dataset.list_files(str(data_root/'*/*'), shuffle=False)
    if directorylevels == 2:
      list_ds = tf.data.Dataset.list_files(str(data_root/'*/*/*'), shuffle=False)
    images_ds = list_ds.map(parse_image)
    #https://stackoverflow.com/questions/70535683/extract-data-from-tensorflow-dataset-e-g-to-numpy
    images = np.asarray(list(images_ds.map(lambda x, y: x)))
    labels = np.asarray(list(images_ds.map(lambda x, y: y)))
    return [images, labels]

# label smoothing function
def smooth_positive_labels(y):
	return y - 0.3 + (random(y.shape) * 0.5)

# select real samples
def generate_real_samples(dataset, n_samples):
	# split into images and labels
	images, labels = dataset
	# choose random instances
	ix = randint(0, images.shape[0], n_samples)
	# select images and labels
	X, labels = images[ix], labels[ix]
	# generate class labels
	y = ones((n_samples, 1))
	# apply label smoothing
	y = smooth_positive_labels(y)
	return [X, labels], y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=10):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	z_input = x_input.reshape(n_samples, latent_dim)
	# generate labels
	labels = randint(0, n_classes, n_samples)
	return [z_input, labels]

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	z_input, labels_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	images = generator([z_input, labels_input])
	# convert output to numpy array
	images = images.numpy()
	# create class labels
	y = zeros((n_samples, 1))
	return [images, labels_input], y

In [14]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
	# prepare fake examples
	[X, _], _ = generate_fake_samples(g_model, latent_dim, n_samples)
	# scale from [-1,1] to [0,1]
	X = (X + 1) / 2.0
	# plot images
	for i in range(100):
		# define subplot
		pyplot.subplot(10, 10, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(X[i])
	# save plot to file
	filename1 = '/content/gdrive/My Drive/planegan/sample_output/generated_plot_e%03d.png' % (step+1)
	pyplot.savefig(filename1)
	pyplot.close()
	# save the generator model
	filename2 = '/content/gdrive/My Drive/planegan/saved_models/generator_model_%03d.h5' % (step+1)
	g_model.save(filename2)
	print('>Saved: %s and %s' % (filename1, filename2))
 
# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist):
	# plot loss
		pyplot.subplot(2, 1, 1)
		pyplot.plot(d1_hist, label='d-real')
		pyplot.plot(d2_hist, label='d-fake')
		pyplot.plot(g_hist, label='gen')
		pyplot.legend()
	# plot discriminator accuracy
		pyplot.subplot(2, 1, 2)
		pyplot.plot(a1_hist, label='acc-real')
		pyplot.plot(a2_hist, label='acc-fake')
		pyplot.legend()
	# save plot to file
		pyplot.savefig('/content/gdrive/My Drive/planegan/saved_models/plot_line_plot_loss.png')
		pyplot.close()
	
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=300, n_batch=128):
	# calculate the number of batches per training epoch
	bat_per_epo = int(dataset[0].shape[0] / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# calculate the size of half a batch of samples
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_steps):
		# get randomly selected 'real' samples
		[X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
		# update discriminator model weights
		_,d_r1,d_r2 = d_model.train_on_batch(X_real, [y_real, labels_real])
		# generate 'fake' examples
		[X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		# update discriminator model weights
		_,d_f,d_f2 = d_model.train_on_batch(X_fake, [y_fake, labels_fake])
		# prepare points in latent space as input for the generator
		[z_input, z_labels] = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = ones((n_batch, 1))
		# update the generator via the discriminator's error
		_,g_1,g_2 = gan_model.train_on_batch([z_input, z_labels], [y_gan, z_labels])
		# summarize loss on this batch
		print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]' % (i+1, d_r1,d_r2, d_f,d_f2, g_1,g_2))
		# evaluate the model performance every 'epoch'
		if (i+1) % (bat_per_epo * 10) == 0:
			summarize_performance(i, g_model, latent_dim)

In [ ]:
# load image data
image_path = "/content/gdrive/My Drive/planegan/airplanes_resized/"
resizedim = 128
directorylevels = 1
dataset = load_custom_image_set(image_path,resizedim,directorylevels)

In [ ]:
# Setup the subplot formatting 
fig, ax = pyplot.subplots(ncols=8, figsize=(20,20))
# Loop four times and get images 
for idx in range(8): 
    index = randint(0, len(dataset[0]))
    # Grab an image and label
    image = (dataset[0][index] + 1) / 2.0
    label = dataset[1][index]
    # Plot the image using a specific subplot 
    ax[idx].imshow(image)
    # Appending the image label as the plot title 
    ax[idx].title.set_text(label)

In [ ]:
# size of the latent space
latent_dim = 100
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, dataset, latent_dim)